In [1]:
import pickle
import numpy as np
import pandas as pd
import datetime
from collections import defaultdict
from scipy import stats, sparse
from scipy.special import expit
from sklearn.preprocessing import OneHotEncoder
from sklearn.linear_model import LogisticRegression, SGDClassifier
from tqdm import tqdm

import warnings
warnings.filterwarnings('ignore')

# 1. Подготовка данных

In [2]:
with open('players.pkl', 'rb') as file1, open('results.pkl', 'rb') as file2, open('tournaments.pkl', 'rb') as file3:
    players_dict = pickle.load(file1)
    results_dict = pickle.load(file2)
    tournaments_dict = pickle.load(file3)

Сначала отберем турниры, по дате проведения и типу.

In [3]:
REQUIRED_TYPES = ["Обычный", "Синхрон", "Строго синхронный"]

In [4]:
train_ids = [key for 
             key, value in tournaments_dict.items()
             if (value['dateStart'][:4] == '2019') and
             (value['type']['name'] in REQUIRED_TYPES)]

test_ids = [key for 
            key, value in tournaments_dict.items()
            if (value['dateStart'][:4] == '2020') and
            value['type']['name'] in REQUIRED_TYPES]

print(f'Число турниров в обучающей выборке: {len(train_ids)}')
print(f'Число турниров в тестовой выборке: {len(test_ids)}')

Число турниров в обучающей выборке: 622
Число турниров в тестовой выборке: 362


Теперь сформируем словарь, который в дальнейшем будем использовать для формирования датасетов. Он будет иметь следующий вид:

```
{
    'tournamet1_id': {
        'team1_id': {
            'mask': ...,
            'players': ...
        },
        
        'team2_id': {
            ...
        },
        ...
    },
    
    'tournamet2_id': {
        ...
    }
}
```

Параллельно выбросим те турниры (команды), для которых либо нет информации о повопросных результатах, либо вовсе отсутствуют результаты.

In [5]:
def form_results(ids):
    data = defaultdict()
    for idx in ids:
        tournamet_result = defaultdict()
        for i, team in enumerate(results_dict[idx]):
            mask = team['mask']
            if mask is not None:
                mask = [0 if i in ["X", "?"] else int(i) for i in team['mask']]
                team_members = [player['player']['id'] for player in team['teamMembers']]
                team_id = team['team']['id']
                tournamet_result[team_id] = defaultdict()
                tournamet_result[team_id]['mask'] = mask
                tournamet_result[team_id]['players'] = team_members
                tournamet_result[team_id]['position'] = i + 1
        if tournamet_result:
            data[idx] = tournamet_result
    return data

In [6]:
results_train = form_results(train_ids)
results_test = form_results(test_ids)

print(f'Число турниров в обучающей выборке: {len(results_train.keys())}')
print(f'Число турниров в тестовой выборке: {len(results_test.keys())}')

Число турниров в обучающей выборке: 612
Число турниров в тестовой выборке: 151


Число турниров изменилось, в тесте значительно уменьшилось. Скорее всего это связано с отсутсвием информации о повопросных результатах.

# 2. Baseline. Рейтинг игроков

Сформируем датасет, состоящий из пар ***player_id-question_id*** с таргетом для этой пары - ответ на вопрос верный/неверный.

***question_id*** сформируем как ***tournament_id + question_id_in_tournament***.

In [7]:
def data_from_results(results):
    data = defaultdict(list)
    for tournament_id, tournament in results.items():
        for team_id, team in tournament.items():
            mask = team['mask']
            questions_ids = [f"{tournament_id}_{i}" for i in range(len(mask))]
            team_ids = [team_id for _ in range(len(mask))]
            tournament_ids = [tournament_id for _ in range(len(mask))]
            for player in team['players']:
                player_ids = len(mask) * [player]
                data['tournament_id'].extend(tournament_ids)
                data['team_id'].extend(team_ids)
                data['players'].extend(player_ids)
                data['questions'].extend(questions_ids)
                data['target'].extend(mask)
    data_df = pd.DataFrame(data)
    return data_df

In [27]:
train_data = data_from_results(results_train)
test_data = data_from_results(results_test)

In [28]:
print(f"Число пар в обучающей выборке: {train_data.shape[0]}")
print(f"Число пар в тестовой выборке: {test_data.shape[0]}")

Число пар в обучающей выборке: 14661596
Число пар в тестовой выборке: 3513640


Для обучения логистичекой регрессии необходимо каким-то образом преобразовать эти данные. Применим OneHotEncoder, т.е. получим свой вектор для каждого ***player_id*** и ***question_id***. 

In [10]:
encoder = OneHotEncoder(handle_unknown='ignore')
encoder.fit(train_data[['players', 'questions']]) 
X_train = encoder.transform(train_data[['players', 'questions']])
y_train = train_data.target.values
unique_players = encoder.categories_[0]

In [11]:
print(f"Размерность получившихся векторов: {X_train.shape[1]}")
print(f"Число уникальных игроков: {unique_players.shape[0]}")
print(f"Число уникальных вопросов: {encoder.categories_[1].shape[0]}")

Размерность получившихся векторов: 73911
Число уникальных игроков: 44729
Число уникальных вопросов: 29182


In [12]:
clf = LogisticRegression(max_iter=100, solver='lbfgs', n_jobs=-1, random_state=0).fit(X_train, y_train)

In [13]:
print(f"Точность модели на обучающей выборке: {clf.score(X_train, y_train) * 100:.2f} %")

Точность модели на обучающей выборке: 77.21 %


Качество модели достаточно неплохое. Однако мы не можем использовать данную модель для предсказаний на тесте, т.к. мы не знаем сложность вопросов будущих турниров. Из полученной модели мы можем извлечь весовые коэффициенты, которые будут представлять из себя силу игрока и сложность вопроса. Так как модель предсказывает вероятность успеха, то  логично предположить, что чем больше вес, характеризующий силу игрока, тем сильнее игрок, для сложности вопроса наоборот, чем сложнее вопрос, тем вес при нем ниже, так как вероятность правильно ответить ниже.

Отранжируем игроков на основе этих весов.

In [14]:
weights = clf.coef_[0][:len(unique_players)]
players_names = [f"{players_dict[player_id]['surname']} {players_dict[player_id]['name']}" 
                 for player_id in unique_players]
trained_rating = pd.DataFrame({'player_id': unique_players, 'rating': weights, 'name': players_names})
trained_rating = trained_rating.sort_values(by='rating', ascending=False)

In [15]:
trained_rating.head(30)

,player_id,rating,name
3800,27403,3.765749,Руссо Максим
598,4270,3.678808,Брутер Александра
4170,30152,3.520882,Сорожкин Артём
3986,28751,3.452479,Семушин Иван
3871,27822,3.361047,Савченков Михаил
4190,30270,3.317436,Спешков Сергей
2889,20691,3.161946,Мереминский Станислав
4718,34328,3.111326,Царёв Михаил
2536,18036,3.108858,Левандовский Михаил
6592,56647,3.099862,Горелова Наталья


Если взглянуть на топ, то можно увидеть достаточно известных знатоков таких, как Сорожкин, Бруттер, Руссо, Николенко и тд, которые на данный момент входят в топ 30-40.

# 3. Предсказание результатов турниров.

Будем ранжировать команды по вероятности ответить хотя бы на один вопрос. Будем считать, что команда ответила верно, если хотя бы один игрок ответил верно. Пусть $p_{ij}$ - вероятность того, что игрок $i$ даст верный ответ на вопрос $j$, тогда вероятность, что ни один игрок не даст ни одного верного ответа:

$$
p_{zero\_answers} = \prod_{i=1}^n \prod_{j=1}^m(1 - p_{ij})
$$

Вероятность, что команда даст хотя бы один верный ответ:

$$
p = 1 - p_{zero\_answers}
$$

Чем выше вероятность ответить хотя бы на один вопрос (чем ниже вероятность дать 0 правильных ответов), тем вышу рейтинг команды.

Так как мы имеем рейтинг только для множества игроков в обучающей выборке, выбросим из тестовой выборки игроков, которых нет в обучающей. 

Также заменим все onehot вектора вопросов на 0, так как сложности новых вопросов мы не имеем. Можно легко заметить, что, так как мы не знаем сложности вопросов, для каждого участника мы будем получать один и тот же скор независимо от вопроса, поэтому вероятность, описанную выше, можно оценивать не по всем вопросам, а только по одному.

In [16]:
test_data = test_data[test_data['players'].isin(unique_players)]
test_data['questions'] = 'unknown'
test_data.head()

,tournament_id,team_id,players,questions,target
0,4957,49804,30152,unknown,1
1,4957,49804,30152,unknown,1
2,4957,49804,30152,unknown,1
3,4957,49804,30152,unknown,1
4,4957,49804,30152,unknown,1


In [17]:
def get_correllations(data, predictions):
    data['predictions'] = predictions

    # Оставим один скор для каждого игрока в команде, так как он не зависит от вопроса
    rating = data.drop_duplicates(['tournament_id', 'team_id', 'players', 'predictions'])
    rating['pred_score'] = (rating
                            .groupby(['tournament_id', 'team_id'])['predictions']
                            .transform(lambda x: 1 - np.prod(1 - x)))
    rating = rating[['tournament_id', 'team_id', 'pred_score']].drop_duplicates()

    # Добавляем реальные позиции команд в турнире
    rating['real_rating'] = rating[['tournament_id', 'team_id']].apply(
                                lambda x: results_test[x.tournament_id][x.team_id]['position'],
                                axis=1
                            )

    # Добавляем предсказанные позиции команд
    rating = rating.sort_values(by=['tournament_id', 'pred_score'], ascending=False)
    rating['pred_rating'] = (rating
                             .groupby('tournament_id')['pred_score']
                             .transform(lambda x: np.arange(1, len(x) + 1)))
    spearman = (rating
            .groupby('tournament_id')
            .apply(lambda x: stats.spearmanr(x.real_rating, x.pred_rating).correlation)
            .mean())

    kendall = (rating
                .groupby('tournament_id')
                .apply(lambda x: stats.kendalltau(x.real_rating, x.pred_rating).correlation)
                .mean())
    
    print(f'Корреляция Спирмена: {spearman:.4f}')
    print(f'Корреляция Кендалла: {kendall:.4f}')

Мы получили рейтинги на основе нашей бейзлайн модели, теперь сравним их с реальными, используя корреляции Спирмена и Кендалла по каждому турниру, затем усредним их.

In [18]:
X_test = encoder.transform(test_data[['players', 'questions']])
predictions = clf.predict_proba(X_test)[:, 1]
get_correllations(test_data, predictions)

Корреляция Спирмена: 0.7964
Корреляция Кендалла: 0.6257


# 4. EM алгоритм.

До этого мы предполагали, что, если команда ответила на вопрос, то и каждый из участников дал верный ответ. Однако в действительности мы не знаем, кто конкретно дал верный ответ. Единственное, что мы можем предполагать, так это то, что, если команда дала неверный ответ, то никто из игроков не дал верный ответ. Таким образом, получаем задачу со скрытой переменной, назовем ее $z_{ij}$, которая означает, что участник $i$ дал верный ответ на вопрос $j$.

Для этой переменной существуют следующие ограничения:
- $z_{ij}$ = 0 для всех $i \in team$, если команда дала неверный ответ
- $z_{ij}$ = 1 хотя бы для одного $i \in team$, если команда дала верный ответ

На Е-шаге алгоритма мы должны оценить ожидание $z_{ij}$. Получаем следующую систему:


$$
\begin{equation}
    \mathbb{E} [z_{ij}]= 
    \begin{cases}
      0, \text{ if } x_{tj} = 0\\
      p(z_{ij}=1| \exists i'\in team, z_{i'j} = 1), \text{ if } x_{tj} = 1
    \end{cases}\,
\end{equation}
$$
, где $x_{tj}$ означает, дала ли камонда $t$ верный ответ на вопрос $j$.

Вероятность легко оценить:

$$
p(z_{ij}=1| \exists i'\in team, z_{i'j} = 1) = \frac{p(z_{ij}=1)}{p(\exists i'\in team, z_{i'j} = 1)} = \frac{\sigma(c_{ij})}{1 - \prod_{i \in team}\sigma(c_{ij})}
$$
, где $c_{ij}$ - вектор, характеризующий силу игрока $i$ и сложность вопроса $j$.

На М-шаге мы, так же как и в случае с baseline моделью обучаем логистическую регрессию, но уже на таргеты, полученные на Е-шаге. Важно отметить, что логистическая регрессия в sklearn не может работать с вероятностями в качестве таргета, поэтому придется написать ее вручную.

In [19]:
class EM():
    def __init__(self, in_dim, lr=15, batch_size=5000, tol=1e-6):
        self.W = np.random.random((1, in_dim + 1))
        self.lr = lr
        self.batch_size = batch_size
        self.tol = tol
        
    def _e_step(self, data, predictions):
        tmp = data[['team_id', 'questions', 'target']]
        tmp['predictions'] = 1 - predictions
        tmp['z_denominator'] = (tmp
                                .groupby(['team_id', 'questions'])['predictions']
                                .transform(lambda x: 1 - np.prod(1 - x)))
        tmp['z_denominator'] = 1 - tmp['z_denominator']
        y = np.clip(predictions / tmp['z_denominator'], 0, 1).values
        y[tmp['target'] == 0] = 0
        return y.reshape(-1, 1)
    
    def _m_step(self, X, y):
        for _ in range(50):
            indices = np.random.permutation(np.arange(X.shape[0]))
            i = 0
            curr_loss = 0
            for i in range(0, X.shape[0] // self.batch_size + 1):
                batch_idx = indices[i * self.batch_size: (i + 1) * self.batch_size]
                x_batch, y_batch = X[batch_idx], y[batch_idx]
                grad_W = (self.predict(x_batch) - y_batch).T @ x_batch / X.shape[0]
                self.W -= self.lr * grad_W

    def fit(self, X_train, train_data, X_test, test_data, n_iter=5):
        X_train = sparse.hstack((np.ones((X_train.shape[0], 1)), X_train), format='csr')
        X_test = sparse.hstack((np.ones((X_test.shape[0], 1)), X_test), format='csr')
        for i in tqdm(range(n_iter)):
            prediction = self.predict(X_train)[:, 0]
            y = self._e_step(train_data, prediction)
            self._m_step(X_train, y)
            get_correllations(test_data, self.predict(X_test))

    def predict(self, X):
        if self.W.shape[1] != X.shape[1]:
            X = sparse.hstack((np.ones((X.shape[0], 1)), X), format='csr')
        return expit(X @ self.W.T)
    
    def _loss(self, true, pred):
        return -(true * np.log(pred) + (1 - true) * np.log(1 - pred))

In [35]:
EM_model = EM(X_train.shape[1], lr=25, batch_size=10000)

In [22]:
EM_model.fit(X_train, train_data, X_test, test_data)

 20%|████████████████                                                                | 1/5 [36:14<2:24:57, 2174.32s/it]

Корреляция Спирмена: 0.3585
Корреляция Кендалла: 0.2527


 40%|███████████████████████████████▏                                              | 2/5 [1:11:56<1:47:46, 2155.58s/it]

Корреляция Спирмена: 0.3883
Корреляция Кендалла: 0.2751


 60%|██████████████████████████████████████████████▊                               | 3/5 [1:47:21<1:11:22, 2141.35s/it]

Корреляция Спирмена: 0.4142
Корреляция Кендалла: 0.2961


 80%|████████████████████████████████████████████████████████████████                | 4/5 [2:24:48<36:23, 2183.03s/it]

Корреляция Спирмена: 0.4444
Корреляция Кендалла: 0.3188


100%|████████████████████████████████████████████████████████████████████████████████| 5/5 [3:01:46<00:00, 2181.30s/it]

Корреляция Спирмена: 0.4648
Корреляция Кендалла: 0.3353


Видно, что целевые корреляции на тесте растут, но процесс обучения очень долгий. Для улучшения имеющегося бейзлайна, возьмем в качестве начальной инициализации веса бейзлайн модели и попытаемся их улучшить.

In [20]:
baseline_weights = np.hstack((clf.intercept_, clf.coef_[0])).reshape(1, -1)

In [21]:
EM_model_improved = EM(X_train.shape[1], lr=25, batch_size=10000)
EM_model_improved.W = baseline_weights.copy()

In [22]:
EM_model_improved.fit(X_train, train_data, X_test, test_data)

 20%|████████████████                                                                | 1/5 [35:20<2:21:23, 2120.93s/it]

Корреляция Спирмена: 0.7966
Корреляция Кендалла: 0.6261


 40%|███████████████████████████████▏                                              | 2/5 [1:11:02<1:46:39, 2133.15s/it]

Корреляция Спирмена: 0.7965
Корреляция Кендалла: 0.6261


 60%|██████████████████████████████████████████████▊                               | 3/5 [1:45:21<1:09:58, 2099.34s/it]

Корреляция Спирмена: 0.7965
Корреляция Кендалла: 0.6261


 80%|████████████████████████████████████████████████████████████████                | 4/5 [2:19:32<34:40, 2080.24s/it]

Корреляция Спирмена: 0.7965
Корреляция Кендалла: 0.6261


100%|████████████████████████████████████████████████████████████████████████████████| 5/5 [2:56:14<00:00, 2114.99s/it]

Корреляция Спирмена: 0.7965
Корреляция Кендалла: 0.6261


ЕМ модель практически не улучшила результаты логистической регрессии. Это может быть связано с тем, что оптимизация свалилась в локальный минимум и уже не смогла оттуда выбраться. 

# 5. Сложность вопросов

Отранжируем турниры по средней сложности вопросов в них. Как отмечалось выше чем сложнее вопрос, тем меньше его вес в модели.

In [23]:
unique_qusetions = encoder.categories_[1]
em_weights = EM_model_improved.W[0]
q_difficulty = {question: difficulty 
                for question, difficulty in zip(unique_qusetions, em_weights[-len(unique_qusetions):])}
train_data['difficulty'] = train_data['questions'].map(q_difficulty)
train_data_question_difficulty = (train_data
                                  .drop_duplicates(['tournament_id', 'questions'])[
                                      ['tournament_id', 'questions', 'difficulty']
                                  ])
train_data_question_difficulty = (train_data_question_difficulty
                                  .groupby(['tournament_id'], as_index=False)['difficulty']
                                  .mean()
                                  .sort_values('difficulty'))
tournament_names = {idx: tournaments_dict[idx]['name']
                    for idx in train_data_question_difficulty.tournament_id}
train_data_question_difficulty['name'] = train_data_question_difficulty['tournament_id'].map(tournament_names)

Посмотрим на топ самых сложных турниров. В топе достаточно много крупных чемпионатов (чемпионаты мира, городов, стран и тд)


In [24]:
train_data_question_difficulty.head(30)

,tournament_id,difficulty,name
604,6149,-3.237692,Чемпионат Санкт-Петербурга. Первая лига
498,5928,-2.300883,Угрюмый Ёрш
349,5684,-2.056993,Синхрон высшей лиги Москвы
37,5159,-1.993416,Первенство правого полушария
582,6101,-1.718463,Воображаемый музей
7,5025,-1.696638,Кубок городов
268,5587,-1.672000,Записки охотника
20,5083,-1.583844,Ускользающая сова
168,5465,-1.561714,Чемпионат России
357,5693,-1.481556,Знание – Сила VI


Посмотрим на топ самых простых турниров. Среди самых простых турниров мы видим чемпионаты вузов, школ и тд.

In [25]:
train_data_question_difficulty.tail(30)

,tournament_id,difficulty,name
274,5593,1.014022,Межфакультетский кубок МГУ. Отбор №3
593,6122,1.037040,Гран-при Бауманки. 2 этап. Кубок весей
101,5391,1.047850,Чемпионат Минска. Лига Б. Тур четвёртый
327,5654,1.054008,Первое зеркало ЮЧЕ
280,5601,1.076979,Межфакультетский кубок МГУ. Отбор №4
338,5667,1.078456,Командум
319,5645,1.084874,Школьники - взрослым
450,5854,1.107095,Лига вузов. III тур
19,5078,1.114934,Синхрон-lite. Выпуск XXV
395,5771,1.125614,Кубок кабачков


Можем заключить, что модель в принципе справилась с оценкой сложности вопросов, так как в топ по сложности попали чемпионаты серьезного уровня, в то время как в числе простых оказались школьные и вузовские этапы.